1. Setup and configuration

In [ ]:
import pandas as pd
import nemosis
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import datetime
import os

# --- Configuration ---
# Set the region and the time period for the analysis.
REGION = 'SA1'
START_DATE = datetime.date(2024, 7, 1)
END_DATE = datetime.date(2025, 6, 30)

# Path for caching the raw data downloaded by NEMOSIS.
RAW_DATA_CACHE_PATH = "./nemosis_cache"

# Ensure the cache directory exists.
if not os.path.exists(RAW_DATA_CACHE_PATH):
    os.makedirs(RAW_DATA_CACHE_PATH)

print("Configuration is set.")

2. Data acquisition with NEMOSIS

In [ ]:
print(f"Fetching dispatch price data for {REGION} from {START_DATE} to {END_DATE}...")

# Use NEMOSIS to download and compile the data.
# This might take a few minutes, especially on the first run as it needs to download the files.
df_raw = nemosis.dynamic_data_compiler(
    start_time=START_DATE.strftime('%Y/%m/%d 00:00:00'),
    end_time=END_DATE.strftime('%Y/%m/%d 23:59:59'),
    table_name='DISPATCHPRICE',
    raw_data_location=RAW_DATA_CACHE_PATH,
    filter_cols=['REGIONID'],
    filter_values=[[REGION]]
)

# Convert the SETTLEMENTDATE to a pandas datetime object for easier manipulation.
df_raw['SETTLEMENTDATE'] = pd.to_datetime(df_raw['SETTLEMENTDATE'])

print(f"Successfully fetched {len(df_raw)} records.")
df_raw.head()

3. Data processing and resampling

In [ ]:
print("Resampling data to 30-minute averages...")

# Set the settlement date as the index for time-series operations.
df_resampled = df_raw.set_index('SETTLEMENTDATE')

# Resample the price data to 30-minute intervals by taking the mean.
df_resampled = df_resampled['RRP'].resample('30min').mean().reset_index()

# Create new columns for the month and the time of day for grouping.
df_resampled['month'] = df_resampled['SETTLEMENTDATE'].dt.to_period('M')
df_resampled['time_of_day'] = df_resampled['SETTLEMENTDATE'].dt.time

print(f"Resampling complete. We now have {len(df_resampled)} 30-minute records.")
df_resampled.head()

4. Calculating decile distributions

In [ ]:
print("Calculating decile distributions for each month and time of day...")

# Define the deciles we want to compute.
deciles = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

# Group by month and time_of_day, then calculate the deciles for the RRP.
df_deciles = df_resampled.groupby(['month', 'time_of_day'])['RRP'].quantile(deciles).unstack()

# Rename the columns to be more descriptive (e.g., '10th', '20th', etc.).
df_deciles.columns = [f'{int(col*100)}th' for col in df_deciles.columns]

# Reset the index so that 'month' and 'time_of_day' become columns again.
df_deciles = df_deciles.reset_index()

# Convert the month period back to a timestamp for easier plotting.
df_deciles['month'] = df_deciles['month'].dt.to_timestamp()


print("Decile calculation complete.")
df_deciles.head()

5. Interactive Visualization with Plotly

In [ ]:
print("Generating interactive visualization...")

# Create the figure
fig = go.Figure()

# Get the list of unique months for the dropdown menu
months = df_deciles['month'].unique()

# Loop through each month to create a separate trace for the visualization
for i, month in enumerate(months):
    df_month = df_deciles[df_deciles['month'] == month]
    month_name = month.strftime('%B %Y')
    
    # Make the traces for the first month visible by default
    is_visible = (i == 0)

    # Add the shaded area for the 10th to 90th percentile (representing the bulk of the distribution)
    fig.add_trace(go.Scatter(
        x=df_month['time_of_day'].astype(str),
        y=df_month['90th'],
        fill=None,
        mode='lines',
        line_color='rgba(0,100,80,0.2)',
        name='10th-90th Percentile',
        visible=is_visible,
        showlegend=is_visible
    ))
    fig.add_trace(go.Scatter(
        x=df_month['time_of_day'].astype(str),
        y=df_month['10th'],
        fill='tonexty', # Fills the area to the previous trace
        mode='lines',
        line_color='rgba(0,100,80,0.2)',
        name='10th-90th Percentile',
        visible=is_visible,
        showlegend=False # Hide legend for the second part of the fill
    ))

    # Add the shaded area for the 30th to 70th percentile
    fig.add_trace(go.Scatter(
        x=df_month['time_of_day'].astype(str),
        y=df_month['70th'],
        fill=None,
        mode='lines',
        line_color='rgba(0,176,246,0.2)',
        name='30th-70th Percentile',
        visible=is_visible,
        showlegend=is_visible
    ))
    fig.add_trace(go.Scatter(
        x=df_month['time_of_day'].astype(str),
        y=df_month['30th'],
        fill='tonexty',
        mode='lines',
        line_color='rgba(0,176,246,0.2)',
        name='30th-70th Percentile',
        visible=is_visible,
        showlegend=False
    ))

    # Add a line for the median (50th percentile)
    fig.add_trace(go.Scatter(
        x=df_month['time_of_day'].astype(str),
        y=df_month['50th'],
        mode='lines',
        line=dict(color='black', width=2),
        name='Median (50th Percentile)',
        visible=is_visible,
        showlegend=is_visible
    ))

# Create the dropdown menu
buttons = []
traces_per_month = 5 # We have 5 traces per month (2 for each shaded area + 1 for the median)

for i, month in enumerate(months):
    visibility = [False] * (len(months) * traces_per_month)
    # Set the traces for the current month to be visible
    visibility[i*traces_per_month : (i+1)*traces_per_month] = [True] * traces_per_month
    
    button = dict(
        label=month.strftime('%B %Y'),
        method='update',
        args=[{'visible': visibility},
              {'showlegend': True}]
    )
    buttons.append(button)

# Update the layout with the dropdown and other formatting
fig.update_layout(
    updatemenus=[dict(
        active=0,
        buttons=buttons,
        direction="down",
        pad={"r": 10, "t": 10},
        showactive=True,
        x=0.1,
        xanchor="left",
        y=1.1,
        yanchor="top"
    )],
    title=f'Monthly Distribution of 30-min Spot Prices in {REGION} ({START_DATE.year}-{END_DATE.year})',
    xaxis_title='Time of Day',
    yaxis_title='Electricity Spot Price ($/MWh)',
    xaxis=dict(tickangle=-45),
    hovermode='x unified'
)


fig.show()